In [16]:
from typing import Literal
from enum import Enum
from pathlib import Path
import json
from ipywidgets import interact, fixed
import pandas as pd
from dataclasses import asdict

from phd_visualizations import save_figure

from solarmed_modeling.fsms import SolarMedState
from solarmed_modeling.fsms.med import FsmParameters as MedFsmParameters
from solarmed_modeling.fsms.sfts import (get_sf_ts_individual_states,
                                         FsmParameters as SfTsFsmParameters)
from solarmed_modeling.fsms.utils import convert_to, SupportedSubsystemsStatesMapping
from solarmed_modeling.visualization.fsm import (
    Node,
    generate_nodes_df,
)
from solarmed_modeling.visualization.fsm.state_evolution import(
    plot_state_graph,
    get_coordinates_edge,
)

from solarmed_optimization.path_explorer.utils import import_results
from solarmed_optimization.visualization import highlight_path

%load_ext autoreload
%autoreload 2

data_path: Path = Path("../results")
n_horizon:int = 5 # 5
sample_time = 1
params_dict: dict[str, dict] = {
    'MED':{
        'sample_time': sample_time,
        **asdict(MedFsmParameters(
            # vacuum_duration_time = 4,
            # brine_emptying_time = 2,
            # startup_duration_time = 2,
            # off_cooldown_time=0,
            # active_cooldown_time=0,

            # vacuum_duration_time = 4,
            # brine_emptying_time = 2,
            # startup_duration_time = 2,
            # off_cooldown_time=9999,
            # active_cooldown_time=5,
            
            vacuum_duration_time = 2,
            brine_emptying_time = 1,
            startup_duration_time = 1,
            off_cooldown_time=9999,
            active_cooldown_time=5,
        )),
    },
    
    'SFTS': {
        'sample_time': sample_time,
        **asdict(SfTsFsmParameters(
        #     recirculating_ts_enabled=True,
        #     idle_cooldown_time = 0,
        
            recirculating_ts_enabled = False,
            idle_cooldown_time = 3,
        )),
    }
    
}


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
paths_sfts_df, _ = import_results(data_path, system="SFTS", n_horizon=n_horizon, params=params_dict["SFTS"])
paths_med_df, _ = import_results(paths_path=data_path, system="MED", n_horizon=n_horizon, params=params_dict["MED"])

display(paths_sfts_df)
# display(paths_sfts_str_df)

nodes_med_df = generate_nodes_df(n_horizon, "MED")
nodes_sfts_df = generate_nodes_df(n_horizon, "SFTS")

display(nodes_sfts_df)


,0,1,2,3,4
0,SfTsState.IDLE,SfTsState.HEATING_UP_SF,SfTsState.IDLE,SfTsState.IDLE,SfTsState.IDLE
1,SfTsState.IDLE,SfTsState.HEATING_UP_SF,SfTsState.SF_HEATING_TS,SfTsState.HEATING_UP_SF,SfTsState.IDLE
2,SfTsState.IDLE,SfTsState.HEATING_UP_SF,SfTsState.SF_HEATING_TS,SfTsState.HEATING_UP_SF,SfTsState.SF_HEATING_TS
3,SfTsState.IDLE,SfTsState.HEATING_UP_SF,SfTsState.SF_HEATING_TS,SfTsState.HEATING_UP_SF,SfTsState.HEATING_UP_SF
4,SfTsState.IDLE,SfTsState.HEATING_UP_SF,SfTsState.SF_HEATING_TS,SfTsState.IDLE,SfTsState.IDLE
...,...,...,...,...,...
57,SfTsState.SF_HEATING_TS,SfTsState.HEATING_UP_SF,SfTsState.HEATING_UP_SF,SfTsState.HEATING_UP_SF,SfTsState.SF_HEATING_TS
58,SfTsState.SF_HEATING_TS,SfTsState.HEATING_UP_SF,SfTsState.HEATING_UP_SF,SfTsState.HEATING_UP_SF,SfTsState.HEATING_UP_SF
59,SfTsState.SF_HEATING_TS,SfTsState.IDLE,SfTsState.IDLE,SfTsState.IDLE,SfTsState.HEATING_UP_SF
60,SfTsState.SF_HEATING_TS,SfTsState.IDLE,SfTsState.IDLE,SfTsState.IDLE,SfTsState.IDLE


,step_idx,state,node_id,state_value,state_name,x_pos,y_pos
0,0,SfTsState.IDLE,step000_0,0,IDLE,0,0.0
1,0,SfTsState.HEATING_UP_SF,step000_1,1,HEATING_UP_SF,0,1.0
2,0,SfTsState.SF_HEATING_TS,step000_2,2,SF_HEATING_TS,0,2.0
3,0,SfTsState.RECIRCULATING_TS,step000_3,3,RECIRCULATING_TS,0,3.0
4,1,SfTsState.IDLE,step001_0,0,IDLE,1,0.0
5,1,SfTsState.HEATING_UP_SF,step001_1,1,HEATING_UP_SF,1,1.0
6,1,SfTsState.SF_HEATING_TS,step001_2,2,SF_HEATING_TS,1,2.0
7,1,SfTsState.RECIRCULATING_TS,step001_3,3,RECIRCULATING_TS,1,3.0
8,2,SfTsState.IDLE,step002_0,0,IDLE,2,0.0
9,2,SfTsState.HEATING_UP_SF,step002_1,1,HEATING_UP_SF,2,1.0


In [18]:
system: Literal["SFTS", "MED"] = "MED"
states_enum: Enum = getattr(SupportedSubsystemsStatesMapping, system).value
paths_df = paths_med_df
nodes_df = nodes_med_df

fig = plot_state_graph(nodes_df, edges_df=None, system_title=system, Np=n_horizon, width=900)
path_values_df = paths_df.map(lambda state: convert_to(state, states_enum, return_format="value"))
max_group_size = path_values_df.groupby("0").size().max()

@interact(initial_state=[state for state in states_enum], option_idx=(0, max_group_size, 1), fig=fixed(fig))
def callback(initial_state=states_enum(0), option_idx=0, fig=fig):
    base_title = f"Directed graph of the operating modes evolution in the {system} system"
    highlight_path(
        fig, paths_df=paths_df, paths_values_df=path_values_df, selected_path_idx=option_idx, 
        initial_state=initial_state, nodes_df=nodes_df, base_title=base_title
    )
    
fig


interactive(children=(Dropdown(description='initial_state', options=(<MedState.OFF: 0>, <MedState.GENERATING_V…

FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(50,50,50)', 'width': 0.5},
              'marker': {'color': '#c061cb', 'size': 20, 'symbol': 'circle-dot'},
              'mode': 'markers',
              'name': 'states',
              'text': array(['OFF', 'GENERATING_VACUUM', 'IDLE', 'STARTING_UP', 'SHUTTING_DOWN',
                             'ACTIVE', 'OFF', 'GENERATING_VACUUM', 'IDLE', 'STARTING_UP',
                             'SHUTTING_DOWN', 'ACTIVE', 'OFF', 'GENERATING_VACUUM', 'IDLE',
                             'STARTING_UP', 'SHUTTING_DOWN', 'ACTIVE', 'OFF', 'GENERATING_VACUUM',
                             'IDLE', 'STARTING_UP', 'SHUTTING_DOWN', 'ACTIVE', 'OFF',
                             'GENERATING_VACUUM', 'IDLE', 'STARTING_UP', 'SHUTTING_DOWN', 'ACTIVE'],
                            dtype=object),
              'type': 'scatter',
              'uid': 'ac281c07-064d-470c-be4e-8e391f6ebfa7',
              'x': [0, 0, 0, 0, 0, 0,